<h1 style='font-size:40px'> Natural Language
Processing with RNNs and
Attention</h1>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O capítulo será composto por duas seções. Na primeira, iremos continuar desenvolvendo nossos estudos com RNN's, mas aplicadas no âmbito de NLP. Já na segunda, iremos dar enfoque aos mecanismos de atenção.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Generating Shakespearean Text Using a
Character RNN</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Aqui, vamos montar uma rede capaz de gerar poemas com o estilo de escrita de Shakespeare. Ela será treinada para prever o próximo caractere a ser digitado.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Creating the Training Dataset</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Antes de tudo, vamos baixar o corpus do projeto e associá-lo a uma variável.
        </li>
    </ul>
 </div>

In [33]:
# Baixando o arquivo com textos do Shakespeare. 
from tensorflow.keras.utils import get_file
file = "shakespeare.txt"
url = "https://homl.info/shakespeare"
filepath = get_file(file, url)

In [34]:
with open(filepath, 'r') as f:
    texts = f.read()

In [35]:
# Para numeralizarmos os textos, vamos recorrer à classe `Tokenizer`. Ao setarmos `char_level=True`, vamos associar cada caractere do corpus
# a um número (começando por 1). 
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([texts])

In [36]:
# Tokenização de caracteres.
tokenizer.texts_to_sequences(['Hi, there!'])

[[7, 6, 18, 1, 3, 7, 2, 9, 2, 31]]

In [37]:
# Observe que o objeto não considera caracteres acentuados, por padrão.
len(tokenizer.texts_to_sequences(['Olá, como vai?'])[0]), len('Ola, como vai?')

(13, 14)

In [38]:
# Convertendo índices em uma string.
tokenizer.sequences_to_texts([[1,2,3,4,5]])

['  e t o a']

In [39]:
# Atribuindo os índices do texto à variável tokenizer. Vamos fazer uma subtração para que o primeiro índice seja atribuído a 0.
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([texts])) - 1
encoded

array([19,  5,  8, ..., 20, 26, 10])

<h3 style='font-size:30px;font-style:italic'> How to Split a Sequential Dataset</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Vamos dividir aqui o nosso corpus no set de treino, validação e teste. O autor aproveita a situação, e revela os prós e contras das diferentes metodologias de separação do dataset. 
        </li>
        <li>
            No nosso caso, vamos optar por manter os primeiros fragmentos do corpus para treinamento, e o restante para validação e teste. Isso tem a assunção de que os mesmos padrões presentes em momentos anteriores do arquivo estarão contidos em momentos futuros (série estacionária).
        </li>
    </ul>
 </div>

In [40]:
from tensorflow.data import Dataset
train_size = encoded.shape[0] * 90 // 100
dataset = Dataset.from_tensor_slices(encoded[:train_size])

<h3 style='font-size:30px;font-style:italic'>Chopping the Sequential Dataset into Multiple Windows</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Até agora, convertemos todo o texto em uma única instância. Como treinar um modelo com isso é inviável, temos que quebrar esses dados em janelas, tratando cada uma delas como uma instância.
        </li>
        <li>
            A função `Dataset.window` cria datasets dentro de nosso dataset principal, cada um contendo uma porção específica de elementos. 
        </li>
    </ul>
 </div>

In [41]:
# Criando sub-datasets de 101 caracteres. Quando definimos `shift=1`, escolhemos que a janela se desloque 1 caractere por vez para montar 
# cada sub-dataset [0-101, 1-102...].

n_steps = 100
window_length = n_steps+1
# `drop_remainder` exclui as últimas janelas cujo tamanho acabaria menor do que `size`.
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

<div> 
    <ul style='font-size:20px'> 
        <li> 
            No entanto, devemos observar que as classes de modelo admitem apenas `tf.Tensor` como input. Caso quisermos converter cada Dataset aninhado em um tensor, podemos fazer um truque com a função `flat_map`, passando a ela uma lambda que cria batches de mesmo comprimento que `window_length`.
        </li>
        <li>
            Em tese, `flat_map` torna Datasets de aninhamento num único Dataset. Mas, como ele admite uma função que aplique alguma transformação nos Datasets aninhados antes da planificação, podemos aplicar `batch` que no final das contas apenas os converta em tensores.
        </li>
    </ul>
 </div>

<p style='color:red'> Vi Creating the Training Dataset e How to Split a Sequential Dataset; Chopping the Sequential Dataset into Multiple Windows</p>

<p style='color:red'> Terminei de ler WaveNet. Começar Cap 16</p>